<a href="https://colab.research.google.com/github/panaku88/MCS-7103-Machine-Learning/blob/main/customer_support_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on last week's lecture, your assignment is to perform Exploratory Data Analysis Process (DAP) on your dataset and write a report that is at least 3 pages long. You can write as many pages as you need. The report needs to be clear and follow a step-by-step process.
1.	Present your question before and after, then the answers. The answers can be before the next step (Data Wrangling), every process or after the full process. The question that helped you select the dataset should differ from the questions you are using for the rest of the process. I have general questions to guide you through your analysis.
2.	Perform in-depth data wrangling.
3.	Provide a well-detailed Exploratory Data Analysis (EDA).
4.	Draw some conclusions based on the performed EDA.
5.	Communicate the ﬁndings in a detailed report.


**INTRODUCTION**

---



Customer satisfaction is an import growth component for any business and in this task, my main objective was to analyze the different touch points between the customer and the different departments of SONIC Company LTD. Such engagements include presales activities/data, technical assistance data, customers care data, and billing queries to find patterns and relationships for improving overall customer services and satisfaction.
What can be done to improve customer satisfaction?

1.   What can be done to improve customer satisfaction?
2.   What kind of information do I need to achieve this?
3.   What processes currently exist?
3.   How do customers reach out to us?







**DATA COLLECTION**

---



The dataset was extracted from the customer management system of SONIC Co. LTD. This includes all logged interactions (queries or complaints) between the customer and sales team, customer and customer care team, customer and billing team, customer and the field support team. The period under consideration is January 2020 to December 2023. Below are the data sources in the dataset.



1.   Presales data – captured presales data include customer requirements
2.   Technical support data – interactions between customers and technical team.
3.   Customer care team – interactions between customer support team members while following up on issues.
4.   Billing queries - records relating to customer billing issues.
After collecting the data, below are the questions that I asked myself to help me in the data analysis
5.   How do we improve customer services?
6.   How do we measure all the customer touch points.
7.   What insight can I learn from the data captured.
8.   What are the most common issues reported by customers?



**DATA WRANGLING**

---



Described in this section are the data pre-processing approach that I used to clean and organize the data. Python was chosen as the data analysis tool.
Important Python libraries: The first step in the data wrangling involved importing the python libraries Pandas, Matplotlib, Seaborn. Pandas is an open-source python data analysis and manipulation tool, Matplotlib is used to perform data visualization in python and lastly seaborn is also a data visualization tool (based on Matplotlib) and is closed integrated with the Pandas data structures making it easier to use, with beautiful statistical graphs.
Reading the Dataset: After extracting the raw customer support dataset, which was extracted in the CSV file format, I loaded the dataset into a Pandas Dataframe. The dataset was downloaded and named as a csv file and then loaded into Pandas data frame for cleaning and exploratory analysis.
Assessing the Dataset: the purpose of this phase was to mainly dig into the dataset and understand the structure, content and check if there are problems in the dataset. The detailed process involved looking at the features attributes in the dataset for instance the Pandas functions head()and dtypes() were useful in understanding the meaning of and datatype of each column in my dataset.
Dataset Cleaning: After understanding the structure and content of the dataset I realized that it was necessary to clean the data as there were some missing values and some attributes that were not important for my purpose. The process involved writing some python code that checked for any null value values and removed them, renamed certain fields as this dataset contains sensitive information.


1. Here I am importing the Google Drive Python Library which is used to connect to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Importing the necessary Python Libraries discussed in the report write up

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import random

3. Reading the raw customer dataset into a Pandas Dataframe and specifying a custom directory (output_path) to store modified dataset

In [ ]:
raw_dataset = pd.read_csv('/content/drive/MyDrive/MCSC1/dataset/CS_Service_Data.csv')

# Specify the path to save the modified/manipulated dataset
output_path = '/content/drive/MyDrive/MCSC1/dataset/customer_support_dataset.csv'


4. Here next code cells, I am assessing the Dataset in order to understand the structure, content and check if there are problems in the dataset

In [ ]:
raw_dataset.head()

In [ ]:
raw_dataset.dtypes

In [ ]:
raw_dataset.info()

In [ ]:
raw_dataset.shape

In [ ]:
raw_dataset.columns

In [ ]:
raw_dataset.nunique()

In [ ]:
raw_dataset.describe()

In [ ]:
raw_dataset.isnull().sum()

5. The following are custom Python code to manipulate the dataset. Precisely, we are removing any sensitive information from the data and any missing or null values.

In [ ]:
customer_names = {}
syllables = ['a', 'e', 'i', 'o', 'u', 'ka', 'ko', 'sa', 'tu', 'ma', 'me', 'mi', 'mo', 'mu', 'ya', 'ye', 'yi', 'yo', 'yu', 'ra', 're', 'ri', 'ro', 'ru', 'wa', 'we', 'wi', 'wo', 'wu']

def generate_name(min_length=3, max_length=6):
  name = ''
  length = random.randint(min_length, max_length)
  for i in range(length):
    name += random.choice(syllables)
  return name.capitalize()

def generate_customer_name(row):
  name = row['CUSTOMER NAME']
  account = row['CUSTOMER ACCOUNT']
  if isinstance(name, str):
    if account in customer_names:
      return customer_names[account]
    else:
      if 'Mr' in name or 'Ms' in name:
        title = random.choice(['Mr', 'Ms'])
        first_name = generate_name()
        last_name = generate_name()
        new_name = f'{title} {first_name} {last_name}'
      elif 'Company' in name or 'Ltd' in name or 'Inc' in name:
        new_name = generate_name() + ' Inc'
      else:
        first_name = generate_name()
        last_name = generate_name()
        new_name = f'{first_name} {last_name}'
      customer_names[account] = new_name
      return new_name
  else:
    return name

raw_dataset['CUSTOMER NAME'] = raw_dataset.apply(generate_customer_name, axis=1)


# Define a function to replace senstive user attributes
def replace_rke_with_tt(text):
    # Check if the value is a string
    if isinstance(text, str):
        # Here i am modifying account information
        return re.sub(r'\bRKE(\w*)\b', r'TT\1', text)
    else:
        # Return the original value if it's not a string
        return text

# Here i am replacing actual service plans
def replace_service_plan(text):
  if isinstance(text, str):
    if re.match(r'CAPPED-BASE: Roke Capped Base', text):
      return 'SONIC HOME PRO 25Mbps'
    else:
      return text
  else:
    return text

# Here i am modifying another service plan
def replace_service_plan_ent(text):
  if isinstance(text, str):
    match = re.search(r'RE(\d+): Roke Enterprise', text)
    if match:
      number = match.group(1)
      return f'SONIC BUSINESS {number}Mbps'
    else:
      return text
  else:
    return text


def replace_service_plan_vpn(text):
  if isinstance(text, str):
    if 'VPN' in text or 'vpn' in text:
      random_number = random.randint(1, 100)
      return f'SONIC MPLS VPN {random_number}Mbps'
    else:
      return text
  else:
    return text

def replace_csnoc(text):
  if isinstance(text, str):
    if text == 'CSNOC':
      return 'SONIC SUPPORT'
    else:
      return text
  else:
    return text



# here i am calling the above functions to manipulate the dataset
raw_dataset['SERVICE PLAN'] = raw_dataset['SERVICE PLAN'].apply(replace_service_plan_vpn)
raw_dataset['SERVICE PLAN'] = raw_dataset['SERVICE PLAN'].apply(replace_service_plan_ent)
raw_dataset['SERVICE PLAN'] = raw_dataset['SERVICE PLAN'].apply(replace_service_plan)
raw_dataset['TICKET NUMBER'] = raw_dataset['TICKET NUMBER'].str.replace(r'^RKE', 'TT', regex=True)
raw_dataset['TICKET CENTER'] = raw_dataset['TICKET CENTER'].apply(replace_csnoc)


6. Data cleanning: After understanding the structure and content in step 5 I realized that it was necessary to clean the data as there were some missing values and some attributes that were not important for my purpose.

In [ ]:
# Save the modified dataset to the specified path
raw_dataset.to_csv(output_path, index=False)

print(f"The clean_cs_dataset.csv has been created successfully at {output_path}.")
new_dataset = pd.read_csv('/content/drive/MyDrive/MCSC1/dataset/customer_support_dataset.csv')

# Drop rows with 'Not Specified' or 'Not Selected'.
new_dataset = new_dataset[new_dataset.applymap(lambda x: 'Not Specified' not in
                                               str(x) and 'Not Selected' not in str(x) and 'Shared Bandwidth' not in str(x)).all(axis=1)]

# Drop the unnecessary attributes
# new_dataset = new_dataset.dropna(subset=['TICKET OWNER'])
# new_dataset = new_dataset.dropna(subset=['SOURCE'])
new_dataset = new_dataset.dropna(subset=['CATEGORY', 'SUB-CATEGORY', 'SOURCE', 'TICKET OWNER'])
new_dataset = new_dataset.drop('CURRENT STATUS', axis=1)
new_dataset = new_dataset.drop('ALLOCATED TIME (HOURS)', axis=1)
new_dataset = new_dataset.drop('LOGGED DESCRIPTION', axis=1)
new_dataset = new_dataset.drop('TICKET OWNER', axis=1)
new_dataset = new_dataset.drop('Logged By User', axis=1)
new_dataset = new_dataset.drop('CUSTOMER ACCOUNT', axis=1)

# Create a dictionary to map source categories into numerical values
source_mapping = {'Email': 1, 'Live Chat': 2, 'Phone': 3, 'Internal Process': 4,
                  'SMS': 5, 'Fax': 6, 'Letter': 7, 'Socials': 8}
new_dataset['SOURCE'] = new_dataset['SOURCE'].map(source_mapping)

# Save the modified dataset to the specified path
new_dataset.to_csv(output_path, index=False)

In [ ]:
new_dataset.info()

In [ ]:
new_dataset.shape

In [ ]:
new_dataset.columns

In [ ]:
new_dataset.head(3)

In [ ]:
new_dataset.tail(3)

In [ ]:
new_dataset.describe()

In [ ]:
new_dataset.isnull().sum()

In [ ]:
new_dataset.dtypes

In [ ]:
new_dataset.nunique()

In [ ]:
new_dataset.duplicated().sum()

**EXPLORATORY DATA ANALYSIS**

---



After performing the data assessment part, I was able to identify certain features that were imported for my purpose.
Some of the important features I identified from the dataset were, the complaint or query category, the time was logged and resolved, service plan of customer, incident id, the customer account with SONIC, whether query came via email, phone or self-logged by user
The features identified for the analysis are total confirmed cases, total recovered cases, total deaths, continents, and population. The reason for choosing these features is simply because we want to know how COVID-19 has affected various parts of the world by comparing confirmed cases with death rate and how people are recovering using their populations.


A Bar plot to investigate the number of incidents logged per service plan

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=new_dataset, x='SUB-CATEGORY')
sns.displot(new_dataset, x="SUB-CATEGORY", hue="CATEGORY")
plt.title('Customer Complaints by Category')
plt.xlabel('Sub-Categories')
plt.ylabel('Customer Complaints')

Investigating the relationship between ticket prioroty and resolution time

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=new_dataset, x='PRIORITY', y='TIME TO CLOSE (MINS)')
plt.title('Relationship between Ticket Priority and Resolution Time')
plt.xlabel('Priority')
plt.ylabel('Resolution Time (Hours)')
plt.show()

A visualization of the number of tickets by ticket source

In [ ]:
# Invert the source_mapping dictionary
inverted_source_mapping = {v: k for k, v in source_mapping.items()}

# Create a new column with categorical values for 'SOURCE'
new_dataset['SOURCE_CATEGORY'] = new_dataset['SOURCE'].map(inverted_source_mapping)

# Create the countplot with hue set to the new column
plt.figure(figsize=(10, 6))
sns.countplot(data=new_dataset, x='SOURCE', hue='SOURCE_CATEGORY')
plt.title('Distribution of Ticket Sources')
plt.xlabel('Ticket Source')
plt.ylabel('Number of Tickets')
plt.legend(title='Ticket Source')
plt.show()

Here I am exploring the relationship between ticket source and resolution time of the tickets using a bax plot

In [ ]:
plt.figure(figsize=(10, 6))

# Create the boxplot without a legend
sns.boxplot(data=new_dataset, x='SOURCE_CATEGORY', y='TIME TO CLOSE (MINS)', showfliers=False)

# Create a custom legend
handles = [plt.Rectangle((0, 0), 1, 1, fc="white", ec="black")] * len(new_dataset['SOURCE_CATEGORY'].unique())
labels = new_dataset['SOURCE_CATEGORY'].unique()
plt.legend(handles, labels, title='Ticket Source')

plt.title('Relationship between Ticket Source and Resolution Time')
plt.xlabel('Ticket Source')
plt.ylabel('Resolution Time (Minutes)')
plt.show()

In this section, explored the relationship between numner of tickets and ticket centers

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=new_dataset, x='TICKET CENTER')
plt.title('Number of Tickets per Ticket Center')
plt.xlabel('Ticket Center')
plt.ylabel('Number of Tickets')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=new_dataset, x='SERVICE PLAN', y='TIME TO CLOSE (MINS)', errorbar=None)
#sns.displot(new_dataset, x="TIME TO CLOSE (MINS)", hue="SERVICE PLAN")
plt.title('Average Resolution Time for Each Service Plan')
plt.xlabel('Service Plan')
plt.ylabel('Average Resolution Time (Hours)')
plt.xticks(rotation=45, ha='right')
plt.show()